In [11]:
!pip install minio requests boto3 --quiet



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [55]:
import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
from datetime import datetime
import os
import sys

In [56]:
try:
    import boto3
    import requests
except ImportError:
    print("Dependencies not found. Please run: pip install minio requests boto3")
    sys.exit(1)

In [71]:

# AI Model Settings
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MODEL_API_URL = f"{INFERENCE_ENDPOINT}/v1/completions"
MODEL_NAME = "granite"

# MinIO Storage Settings
MINIO_ENDPOINT = "minio-api-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
MINIO_BUCKET = "logs"
LOG_FILE = "service_error_down.txt" # This variable will be used throughout the script


In [72]:

## CHECKING MINIO RESOURCES  
def list_files_in_minio(bucket_name):
    """Lists all files in a specified MinIO bucket."""
    print(f"\n--> Checking files in bucket '{bucket_name}'...")
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response and response['Contents']:
            print("    ✅ Files found:")
            for obj in response['Contents']:
                print(f"       - {obj['Key']}")
        else:
            print(f"    ℹ️  Bucket '{bucket_name}' is empty or does not exist.")
        return True
    except Exception as e:
        print(f"    ❌ Error listing files from MinIO: {e}")
        return False

def read_log_from_minio(bucket_name, object_name):
    """Reads a log file's content from a MinIO bucket."""
    print(f"--> Reading '{object_name}' from bucket '{bucket_name}'...")
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        response = s3_client.get_object(Bucket=bucket_name, Key=object_name)
        content = response['Body'].read().decode('utf-8')
        print(f"    ✅ Successfully read log content from MinIO.")
        return content
    except s3_client.exceptions.NoSuchKey:
        print(f"    ❌ Error: The object '{object_name}' was not found in bucket '{bucket_name}'.")
    except Exception as e:
        print(f"    ❌ An error occurred while reading from MinIO: {e}")
    return None

In [73]:

## CHECKING MINIO RESOURCES  
def list_files_in_minio(bucket_name):
    """Lists all files in a specified MinIO bucket."""
    print(f"\n--> Checking files in bucket '{bucket_name}'...")
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response and response['Contents']:
            print("    ✅ Files found:")
            for obj in response['Contents']:
                print(f"       - {obj['Key']}")
        else:
            print(f"    ℹ️  Bucket '{bucket_name}' is empty or does not exist.")
        return True
    except Exception as e:
        print(f"    ❌ Error listing files from MinIO: {e}")
        return False

def read_log_from_minio(bucket_name, object_name):
    """Reads a log file's content from a MinIO bucket."""
    print(f"--> Reading '{object_name}' from bucket '{bucket_name}'...")
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        response = s3_client.get_object(Bucket=bucket_name, Key=object_name)
        content = response['Body'].read().decode('utf-8')
        print(f"    ✅ Successfully read log content from MinIO.")
        return content
    except s3_client.exceptions.NoSuchKey:
        print(f"    ❌ Error: The object '{object_name}' was not found in bucket '{bucket_name}'.")
    except Exception as e:
        print(f"    ❌ An error occurred while reading from MinIO: {e}")
    return None

In [74]:
list_files_in_minio(MINIO_BUCKET)


--> Checking files in bucket 'logs'...
    ✅ Files found:
       - httpd_down.txt
       - service_error.txt
       - service_error_down.txt
       - service_error_up.txt
       - service_error_wentan.txt


True

In [76]:
print(LOG_FILE)

service_error_down.txt


In [77]:
read_log_from_minio(MINIO_BUCKET, LOG_FILE)

--> Reading 'service_error_down.txt' from bucket 'logs'...
    ✅ Successfully read log content from MinIO.


'HTTP SERVER HEALTH REPORT\\n=========================\\n[Basic Information]\\n- Hostname: aiops\\n- OS: RedHat 9.5\\n\\n[Web Server Status]\\n- Installed: Yes\\n- Service Status: UNKNOWN\\n- Listening on Port 80: No\\n\\n[Security Configuration]\\n\\n\\n[Connectivity]\\n- HTTP Response: -1\\n- Config Valid: Valid\\n'

In [80]:

def query_model_for_report(logs):
    """Queries the Granite model with a specific prompt to generate a full incident report."""
    print(f"\n--> Querying AI model for full report...")
    prompt = f"""
Generate a concise, structured AIOps incident report based on the following logs. The service name must be the short, executable name (e.g., 'httpd').
The report must be in Markdown format and include:
1. Key metadata: INCIDENT ID, DETECTED, SEVERITY, STATUS, AFFECTED SERVICE, AFFECTED HOST, SUMMARY, and KEY ERROR LOG.
2. A ROOT CAUSE section explaining the 'why'.
3. A REMEDIATION PLAYBOOK section with exact, numbered shell commands to fix the issue.
4. A VALIDATION section with a command to confirm the fix.

--- LOGS START ---
{logs}
--- LOGS END ---
"""
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": MODEL_NAME, 
        "prompt": prompt,
        "max_tokens": 512 
    }
    
    try:
        response = requests.post(MODEL_API_URL, headers=headers, json=payload, verify=False)
        response.raise_for_status()
        
        result = response.json()
        if 'choices' in result and result['choices']:
            report_text = result['choices'][0].get('text', 'Error: Could not extract report from model response.')
            print("    ✅ Analysis complete.")
            return report_text
        else:
            return f"### Error\nUnexpected response format from model: `{json.dumps(result)}`"
    except Exception as e:
        return f"### Error\nAn unexpected exception occurred: {e}"


In [81]:

# --- Main Execution ---
def main(log_content):
    """Main function to run the AIOps report generation."""
    # Check if the log content is valid before proceeding
    if log_content and log_content.strip():
        incident_report = query_model_for_report(log_content)
        
        print("\n" + "="*50)
        print("     AIOps Incident Remediation Report")
        print("="*50 + "\n")
        print(incident_report)
        
    elif log_content is not None:
        print(f"\n❌ Analysis skipped: The log file '{LOG_FILE}' is empty.")
    else:
        print(f"\n❌ Analysis skipped: Failed to read '{LOG_FILE}' from MinIO.")


if __name__ == "__main__":
    # List files in the bucket for verification first
    list_files_in_minio(MINIO_BUCKET)

    # Read the specified log file from MinIO
    log_content = read_log_from_minio(MINIO_BUCKET, LOG_FILE)
    
    # Print the content of the file that was read
    if log_content:
        print(f"\n--- Content of {LOG_FILE} ---")
        print(log_content)
        print("---------------------------------\n")

    # Now, call the main function to process the log content and generate the report
    main(log_content)



--> Checking files in bucket 'logs'...
    ✅ Files found:
       - httpd_down.txt
       - service_error.txt
       - service_error_down.txt
       - service_error_up.txt
       - service_error_wentan.txt
--> Reading 'service_error_down.txt' from bucket 'logs'...
    ✅ Successfully read log content from MinIO.

--- Content of service_error_down.txt ---
HTTP SERVER HEALTH REPORT\n=========================\n[Basic Information]\n- Hostname: aiops\n- OS: RedHat 9.5\n\n[Web Server Status]\n- Installed: Yes\n- Service Status: UNKNOWN\n- Listening on Port 80: No\n\n[Security Configuration]\n\n\n[Connectivity]\n- HTTP Response: -1\n- Config Valid: Valid\n
---------------------------------


--> Querying AI model for full report...


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


    ✅ Analysis complete.

     AIOps Incident Remediation Report


--- AIOPS INCIDENT REPORT ---

**INCIDENT ID:** INC001
**DETECTED:** 2023-10-12 12:34:56
**SEVERITY:** High
**STATUS:** Ongoing
**AFFECTED SERVICE:** httpd
**AFFECTED HOST:** aiops
**SUMMARY:** The httpd service on the aiops host is in an UNKNOWN state and not listening on port 80.
**KEY ERROR LOG:** HTTP Response: -1

**ROOT CAUSE:**
The httpd service is in an UNKNOWN state, which could be due to a misconfiguration, a failed startup, or a missing package. The fact that it's not listening on port 80 suggests a potential issue with the service configuration or a failed startup.

**REMEDIATION PLAYBOOK:**

1. Check the service status: `systemctl status httpd`
2. Restart the service: `systemctl restart httpd`
3. Verify the service is running: `systemctl is-active httpd`
4. Check if the service is listening on port 80: `ss -tlnp | grep :80`
5. If the service is not listening, update the firewall rules: `firewall-cmd --perma

In [47]:
KEYWORDS_FILE = "extracted_keywords.json"

In [49]:

def read_keywords_from_json(filename):
    """Reads the extracted keywords from a JSON file."""
    print(f"--> Step 1: Reading keywords from '{filename}'...")
    try:
        with open(filename, 'r') as f:
            keywords = json.load(f)
        
        if "affected_host" in keywords and "affected_service" in keywords:
            print(f"    ✅ Successfully read keywords.")
            return keywords
        else:
            print(f"    ❌ Error: JSON file is missing 'affected_host' or 'affected_service' keys.")
            return None
            
    except FileNotFoundError:
        print(f"    ❌ Error: The file '{filename}' was not found. Please run the extraction script first.")
    except Exception as e:
        print(f"    ❌ An error occurred while reading the JSON file: {e}")
    return None

def generate_ansible_playbook(host, service):
    """Generates a diagnostic Ansible playbook in YAML format."""
    print(f"\n--> Step 2: Generating Ansible playbook for {host}...")
    
    # This dictionary structure represents the desired Ansible Playbook format.
    playbook_data = [
        {
            'name': 'Webserver Diagnostic Toolkit',
            'hosts': host,
            'become': True,
            'vars': {
                'webserver_service': service,
                'website_url': f"http://{host}",
                'check_port': 80
            },
            'tasks': [
                {
                    'name': f"Check status of {service} service",
                    'ansible.builtin.service_facts': {}
                },
                {
                    'name': 'Display service status from facts',
                    'ansible.builtin.debug': {
                        'msg': f"The state of {service} is {{'{{'}} ansible_facts.services['{service}.service'].state {{'}}'}}"
                    }
                },
                {
                    'name': 'Check if webserver port is open',
                    'ansible.builtin.wait_for': {
                        'port': "{{ check_port }}",
                        'host': "0.0.0.0",
                        'state': "started",
                        'timeout': 5
                    }
                }
            ]
        }
    ]

    try:
        # Use yaml.dump to convert the Python dictionary to a clean YAML string
        # sort_keys=False preserves the order of the dictionary.
        yaml_output = yaml.dump(playbook_data, sort_keys=False, indent=2)
        print("    ✅ Ansible Playbook generated successfully.")
        return yaml_output
    except ImportError:
        print("    ❌ PyYAML is not installed. Please run 'pip install pyyaml' to generate the playbook.")
        return None
    except Exception as e:
        print(f"   ❌ Error generating YAML: {e}")
        return None


# --- Main Execution ---
def main():
    """Main function to run the AIOps workflow."""
    # 1. Read the keywords from the JSON file
    keywords = read_keywords_from_json(KEYWORDS_FILE)
    if not keywords:
        sys.exit("Pipeline stopped: Could not retrieve keywords.")
        
    # 2. Generate the Ansible playbook using the keywords
    ansible_playbook = generate_ansible_playbook(
        keywords.get("affected_host"), 
        keywords.get("affected_service")
    )
    if not ansible_playbook:
        sys.exit("Pipeline stopped: Could not generate the Ansible playbook.")

    # 3. Print the final, clean report
    print("\n" + "="*50)
    print("      AIOps Generated Ansible Playbook")
    print("="*50 + "\n")
    print(ansible_playbook)


if __name__ == "__main__":
    print("==============================================")
    print("    AIOps PIPELINE: KEYWORDS TO PLAYBOOK")
    print("==============================================")
    main()
    print("\n==============================================")
    print("            PIPELINE COMPLETE")
    print("==============================================")


    AIOps PIPELINE: KEYWORDS TO PLAYBOOK
--> Step 1: Reading keywords from 'extracted_keywords.json'...
    ✅ Successfully read keywords.

--> Step 2: Generating Ansible playbook for aiops.demo.vm...
   ❌ Error generating YAML: name 'yaml' is not defined


SystemExit: Pipeline stopped: Could not generate the Ansible playbook.